### Source Images from Open Adobe Project
[Adobe Panoramas](https://liquidtelecom.dl.sourceforge.net/project/adobedatasets.adobe/adobe_panoramas.tgz)

In [7]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
%config IPCompleter.greedy=True
%config Completer.use_jedi = False

In [35]:
# generating the necessary txt files to input for the super glue algorithm
img_name = 'hotel'
num_images = 8
with open('adobe_panorama.txt', 'w') as file:
    for i in range(num_images -1):
        file.write("{img}-{:02}.png {img}-{:02}.png\n".format(i,i+1, img = img_name))
        
# Running the super glue algorithm on outdoor dataset to match features
!python match_pairs.py  --resize 1600 --superglue outdoor\
                        --max_keypoints 2048 --nms_radius 3\
                        --resize_float \
                        --input_dir adobe_panorama/ \
                        --input_pairs adobe_panorama.txt \
                        --output_dir adobe_panorama/output \
                        --viz --cache 

Namespace(input_pairs='adobe_panorama.txt', input_dir='adobe_panorama/', output_dir='adobe_panorama/output', max_length=-1, resize=[1600], resize_float=True, superglue='outdoor', max_keypoints=2048, keypoint_threshold=0.005, nms_radius=3, sinkhorn_iterations=20, match_threshold=0.2, viz=True, eval=False, fast_viz=False, cache=True, show_keypoints=False, viz_extension='png', opencv_display=False, shuffle=False, force_cpu=False)
Will resize max dimension to 1600
Running inference on device "cuda"
Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)
Looking for data in directory "adobe_panorama"
Will write matches to directory "adobe_panorama\output"
Will write visualization images to directory "adobe_panorama\output"
[Finished pair     0 of     7] load_cache=0.155 total=0.155 sec {6.4 FPS} 
[Finished pair     1 of     7] load_cache=0.112 total=0.112 sec {8.9 FPS} 
[Finished pair     2 of     7] load_cache=0.082 total=0.082 sec {12.2 FPS} 
[Finished pair     3 of     7] load

In [52]:
#generatig the npz files for extract matching information
npz_files = ["{img}-{:02}_{img}-{:02}_matches.npz".format(i,i+1, img = img_name) for i in range(num_images -1)]
for file in npz_files:
    path = 'adobe_panorama/output/'+file
    npz = np.load(path)
print(npz.files)

['keypoints0', 'keypoints1', 'matches', 'match_confidence']


For each keypoint in `keypoints0`, the `matches` array indicates the index of the matching keypoint in `keypoints1`, or `-1` if the keypoint is unmatched.

```python
>>> import numpy as np
>>> path = 'dump_match_pairs/scene0711_00_frame-001680_scene0711_00_frame-001995_matches.npz'
>>> npz = np.load(path)
>>> npz.files
['keypoints0', 'keypoints1', 'matches', 'match_confidence']
>>> npz['keypoints0'].shape
(382, 2)
>>> npz['keypoints1'].shape
(391, 2)
>>> npz['matches'].shape
(382,)
>>> np.sum(npz['matches']>-1)
115
>>> npz['match_confidence'].shape
(382,)
```

In [108]:
# testing with one pair of images 
npz = np.load('adobe_panorama/output/'+npz_files[0])
npz.files
point_set1 = npz['keypoints0']
matching_indexes =  npz['matches'] > -1
point_set2 = [npz['keypoints1']

print(len(matching_points))
H, status = cv.findHomography(point_set1, point_set2)#, cv.RANSAC, 5.0) # computing H to transform right_image to left_image
print(H)
H = np.linalg.inv(H)
H = H/H[-1,-1]
print(H)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 2048 but corresponding boolean dimension is 1809

In [59]:
npz['keypoints0'].shape

(1809, 2)

In [72]:
npz['keypoints1'].shape

(2048, 2)

In [61]:
npz['matches'].shape

(1809,)

In [106]:
match = npz['matches']>-1
match[1000:1200]

array([False,  True, False,  True,  True,  True,  True,  True, False,
       False, False, False, False,  True,  True,  True,  True, False,
        True,  True,  True,  True, False, False,  True, False, False,
       False, False,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True, False, False,
        True, False,  True, False,  True,  True, False,  True, False,
        True,  True,  True,  True, False,  True,  True, False, False,
       False, False, False, False,  True,  True, False,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False,  True,  True, False,  True, False, False, False, False,
       False, False,  True,  True, False, False,  True,  True, False,
       False,  True,

In [105]:
npz['keypoints1'][matching_indexes]

array([[1143.,  404.],
       [1143.,  404.],
       [1143.,  404.],
       ...,
       [1143.,  404.],
       [1143.,  404.],
       [1143.,  404.]], dtype=float32)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed